# PyTorch Practical Exercises

**Topic:** Practical exercises to help solidify the concepts of using PyTorch to build neural networks.
**Total Exercises:** 5
**Estimated Time:** 3-5 hours

---

This notebook was generated from `PyTorch_practical_exercises.md`. Follow the sections in order and run code cells interactively.

## Table of Contents

1. [Environment & Setup](#environment--setup)
2. [Exercise 1: Setting Up PyTorch](#exercise-1-setting-up-pytorch)
3. [Exercise 2: Building a Simple Neural Network](#exercise-2-building-a-simple-neural-network)
4. [Exercise 3: Training the Neural Network](#exercise-3-training-the-neural-network)
5. [Exercise 4: Evaluating the Model](#exercise-4-evaluating-the-model)
6. [Exercise 5: Advanced Practice - CIFAR & Transfer Learning](#exercise-5-advanced-practice---cifar--transfer-learning)
7. [Notes and Next Steps](#notes-and-next-steps)

---

## Learning Objectives

| Objective | Skills Developed |
|---|---|
| Verify PyTorch installation | Importing PyTorch, checking versions and CUDA availability |
| Build simple neural networks | Implementing `nn.Module`, layers and forward pass |
| Train models | Data loading, training loop, optimizer and loss |
| Evaluate models | Compute accuracy on test data |
| Transfer learning | Load pretrained models and fine-tune final layers |

---

## Environment & Setup

Use a virtual environment (venv or conda) and install the appropriate PyTorch build for your platform. For Google Colab, you can run `pip install torch torchvision` if required. The next cell verifies the installation.

## Exercise 1: Setting Up PyTorch

**Task:** Install PyTorch on your local machine or cloud environment and verify the installation by importing PyTorch and checking the version and CUDA availability.

In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

PyTorch version: 2.8.0
CUDA available: False


## Exercise 2: Building a Simple Neural Network

**Task:** Define a simple feedforward neural network using `nn.Module` for MNIST (flattened 28x28 inputs). The example below defines three linear layers (784 -> 128 -> 64 -> 10) and uses ReLU activations.

In [2]:
import torch
import torch.nn as nn


class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input layer
        self.fc2 = nn.Linear(128, 64)  # Hidden layer
        self.fc3 = nn.Linear(64, 10)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=1)
        return x

In [3]:
# Instantiate the network
model = SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


## Exercise 3: Training the Neural Network

**Task:** Train the neural network on a simple dataset such as MNIST. The example training loop below uses CrossEntropyLoss and SGD. Reduce epochs for quick tests.

In [4]:
import torch.optim as optim
from torchvision import datasets, transforms

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
# Training loop
for epoch in range(1, 6):  # 5 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        # Flatten the images into vectors
        images = images.view(images.shape[0], -1)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

    print(f"Epoch {epoch}, Loss: {running_loss/len(trainloader)}")

Epoch 1, Loss: 2.2992332791646324
Epoch 2, Loss: 2.267253607813517
Epoch 3, Loss: 2.1011886843363445
Epoch 4, Loss: 1.8047558176676433
Epoch 5, Loss: 1.6901330506006877


## Exercise 4: Evaluating the Model

**Task:** Evaluate the trained model on a validation/test set and report accuracy. The example below runs inference without gradients and computes accuracy.

In [6]:
testset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images = images.view(images.shape[0], -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 82.81%


## Exercise 5: Advanced Practice - CIFAR & Transfer Learning

**Task:** Implement a CNN for CIFAR-10 and try transfer learning with a pretrained ResNet. Below are examples for loading CIFAR-10 and preparing a pretrained ResNet for fine-tuning.

In [7]:
import torchvision
import torchvision.transforms as transforms

# Define a transformation to apply to the images
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Download and load the training dataset
trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# Download and load the test dataset
testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Classes in CIFAR-10
classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)
print("CIFAR-10 classes:", classes)

CIFAR-10 classes: ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


### Transfer learning (fine-tuning a pretrained ResNet)

Load a pretrained ResNet, replace the final fully connected layer for 10 classes, and (optionally) freeze earlier layers. Ensure inputs are resized to 224x224 if using the standard pretrained weights.

In [8]:
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load pretrained ResNet18
resnet = models.resnet18(pretrained=True)
# Freeze pretrained parameters (optional)
for param in resnet.parameters():
    param.requires_grad = False

# Replace the final layer to match 10 classes
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)
resnet = resnet.to(device)

# Only train the final layer
optimizer = optim.SGD(resnet.fc.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

print(resnet)

# Note: If using CIFAR images (32x32), consider upsampling to 224x224
# or adapt the model input accordingly.

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

/Users/nev4rb14su/.conda/envs/lab/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nev4rb14su/.conda/envs/lab/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Notes and Next Steps

- For quick experiments set small epochs and batch sizes; increase them for final runs.
- Add model checkpointing (torch.save) and evaluation logging for reproducibility.
- Extend the notebook with visualization of training/validation loss and accuracy.
- If you want, I can split this notebook into smaller cells with more comments or add full training scripts for transfer learning.